In [ ]:
# Install neqsim if needed (uncomment for Colab)
# !pip install neqsim

In [ ]:
from neqsim.thermo import fluid, TPflash, PHflash
from neqsim import jneqsim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## 1. Create Reservoir Fluid

We'll use a typical black oil composition for the PVT experiments.

In [ ]:
# Create black oil with plus fraction
oil = fluid("pr")
oil.addComponent("nitrogen", 0.5)
oil.addComponent("CO2", 1.5)
oil.addComponent("methane", 45.0)
oil.addComponent("ethane", 8.0)
oil.addComponent("propane", 5.0)
oil.addComponent("i-butane", 1.5)
oil.addComponent("n-butane", 3.0)
oil.addComponent("i-pentane", 1.5)
oil.addComponent("n-pentane", 2.0)
oil.addComponent("n-hexane", 3.0)
oil.addComponent("C7", 8.0, 95.0, 0.72)   # mole%, MW, density
oil.addComponent("C8", 6.0, 107.0, 0.75)
oil.addComponent("C9", 4.0, 121.0, 0.78)
oil.addComponent("C10", 3.0, 134.0, 0.79)
oil.addPlusFraction("C11+", 8.0, 250.0, 0.85)  # Plus fraction
oil.setMixingRule(2)
oil.setMultiPhaseCheck(True)

# Reservoir conditions
T_reservoir = 100.0  # °C
P_reservoir = 350.0  # bara

oil.setTemperature(T_reservoir, "C")
oil.setPressure(P_reservoir, "bara")
TPflash(oil)

print(f"Reservoir conditions: {P_reservoir} bara, {T_reservoir}°C")
print(f"Number of phases: {oil.getNumberOfPhases()}")
print(f"This fluid is: {'undersaturated (single-phase)' if oil.getNumberOfPhases() == 1 else 'saturated (two-phase)'}")

## 2. Find Saturation Pressure (Bubble Point)

Before running PVT experiments, we need to find the bubble point pressure.

In [ ]:
# Calculate saturation pressure
ops = jneqsim.thermodynamicoperations.ThermodynamicOperations(oil)
oil.setTemperature(T_reservoir, "C")
oil.setPressure(P_reservoir, "bara")

try:
    ops.calcSaturationPressure()
    P_sat = oil.getPressure()
    print(f"Bubble point pressure at {T_reservoir}°C: {P_sat:.1f} bara")
except:
    # Estimate by scanning
    P_sat = 200.0  # Use approximate value
    print(f"Estimated bubble point: ~{P_sat} bara")

## 3. Constant Composition Expansion (CCE)

In CCE, the fluid is expanded at constant temperature while measuring:
- Relative volume (V/V_sat)
- Compressibility
- Liquid dropout (below bubble point)

In [ ]:
# Setup CCE experiment
cce = jneqsim.pvtsimulation.simulation.ConstantMassExpansion(oil)
cce.setTemperature(T_reservoir, "C")

# Pressure steps
pressures = np.linspace(350, 50, 31)  # From 350 to 50 bara
cce.setPressures(pressures.tolist(), "bara")

# Run experiment
cce.runCalc()

print("CCE experiment completed")

In [ ]:
# Get CCE results
cce_pressures = list(cce.getPressures())
relative_volumes = list(cce.getRelativeVolume())
liquid_fractions = list(cce.getLiquidVolume())
z_factors = list(cce.getZMix())

# Create results DataFrame
cce_df = pd.DataFrame({
    'Pressure [bara]': cce_pressures,
    'Relative Volume': relative_volumes,
    'Liquid Vol %': [lv * 100 for lv in liquid_fractions],
    'Z-factor': z_factors
})

print("CCE Results (sample):")
print(cce_df.head(15).to_string(index=False))

In [ ]:
# Plot CCE results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Relative volume
ax1.plot(cce_pressures, relative_volumes, 'b-o', linewidth=2, markersize=4)
ax1.axvline(x=P_sat, color='r', linestyle='--', label=f'Bubble Point ({P_sat:.0f} bara)')
ax1.set_xlabel('Pressure [bara]', fontsize=12)
ax1.set_ylabel('Relative Volume V/V_sat', fontsize=12)
ax1.set_title('CCE: Relative Volume', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.invert_xaxis()

# Liquid volume percent
ax2.plot(cce_pressures, [lv * 100 for lv in liquid_fractions], 'g-s', linewidth=2, markersize=4)
ax2.axvline(x=P_sat, color='r', linestyle='--', label=f'Bubble Point ({P_sat:.0f} bara)')
ax2.set_xlabel('Pressure [bara]', fontsize=12)
ax2.set_ylabel('Liquid Volume [%]', fontsize=12)
ax2.set_title('CCE: Liquid Dropout', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.invert_xaxis()

plt.tight_layout()
plt.show()

## 4. Differential Liberation (DL)

In DL, gas is removed at each pressure step, simulating the reservoir depletion process:
- Measures Rs (solution GOR)
- Measures Bo (oil formation volume factor)
- Gives residual oil properties

In [ ]:
# Create fresh fluid for DL
oil_dl = oil.clone()
oil_dl.setTemperature(T_reservoir, "C")
oil_dl.setPressure(P_reservoir, "bara")

# Setup DL experiment
dl = jneqsim.pvtsimulation.simulation.DifferentialLiberation(oil_dl)
dl.setTemperature(T_reservoir, "C")

# Pressure steps (start from bubble point and go down)
dl_pressures = np.linspace(P_sat, 1.0, 25)
dl.setPressures(dl_pressures.tolist(), "bara")

# Run experiment
dl.runCalc()

print("Differential Liberation completed")

In [ ]:
# Get DL results
dl_p = list(dl.getPressures())
Bo = list(dl.getBo())
Rs = list(dl.getRs())
oil_density = list(dl.getOilDensity())

dl_df = pd.DataFrame({
    'Pressure [bara]': dl_p,
    'Bo [m3/Sm3]': Bo,
    'Rs [Sm3/Sm3]': Rs,
    'Oil Density [kg/m3]': oil_density
})

print("Differential Liberation Results:")
print(dl_df.to_string(index=False))

In [ ]:
# Plot DL results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Bo
axes[0, 0].plot(dl_p, Bo, 'b-o', linewidth=2, markersize=4)
axes[0, 0].set_xlabel('Pressure [bara]', fontsize=11)
axes[0, 0].set_ylabel('Bo [m³/Sm³]', fontsize=11)
axes[0, 0].set_title('Oil Formation Volume Factor (Bo)', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].invert_xaxis()

# Rs
axes[0, 1].plot(dl_p, Rs, 'g-s', linewidth=2, markersize=4)
axes[0, 1].set_xlabel('Pressure [bara]', fontsize=11)
axes[0, 1].set_ylabel('Rs [Sm³/Sm³]', fontsize=11)
axes[0, 1].set_title('Solution Gas-Oil Ratio (Rs)', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].invert_xaxis()

# Oil Density
axes[1, 0].plot(dl_p, oil_density, 'r-^', linewidth=2, markersize=4)
axes[1, 0].set_xlabel('Pressure [bara]', fontsize=11)
axes[1, 0].set_ylabel('Density [kg/m³]', fontsize=11)
axes[1, 0].set_title('Oil Density', fontsize=12)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].invert_xaxis()

# Bo vs Rs crossplot
axes[1, 1].plot(Rs, Bo, 'm-d', linewidth=2, markersize=4)
axes[1, 1].set_xlabel('Rs [Sm³/Sm³]', fontsize=11)
axes[1, 1].set_ylabel('Bo [m³/Sm³]', fontsize=11)
axes[1, 1].set_title('Bo vs Rs', fontsize=12)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Constant Volume Depletion (CVD)

CVD simulates gas condensate reservoir depletion:
- Fixed cell volume (original reservoir volume)
- Gas removed to maintain constant volume
- Measures retrograde liquid dropout

In [ ]:
# Create gas condensate fluid for CVD
condensate = fluid("pr")
condensate.addComponent("nitrogen", 1.0)
condensate.addComponent("CO2", 2.0)
condensate.addComponent("methane", 70.0)
condensate.addComponent("ethane", 8.0)
condensate.addComponent("propane", 5.0)
condensate.addComponent("i-butane", 2.0)
condensate.addComponent("n-butane", 3.0)
condensate.addComponent("i-pentane", 2.0)
condensate.addComponent("n-pentane", 2.0)
condensate.addComponent("n-hexane", 2.0)
condensate.addComponent("C7", 3.0, 95.0, 0.72)
condensate.setMixingRule(2)
condensate.setMultiPhaseCheck(True)

T_cond = 90.0  # °C
P_cond = 400.0  # bara
condensate.setTemperature(T_cond, "C")
condensate.setPressure(P_cond, "bara")

print(f"Gas condensate at {P_cond} bara, {T_cond}°C")

In [ ]:
# Setup CVD experiment
cvd = jneqsim.pvtsimulation.simulation.ConstantVolumeDepletion(condensate)
cvd.setTemperature(T_cond, "C")

cvd_pressures = np.linspace(400, 50, 25)
cvd.setPressures(cvd_pressures.tolist(), "bara")

# Run
cvd.runCalc()

print("CVD experiment completed")

In [ ]:
# Get CVD results
cvd_p = list(cvd.getPressures())
liquid_dropout = list(cvd.getLiquidVolume())
z_two_phase = list(cvd.getZMix())

# Plot retrograde liquid dropout
plt.figure(figsize=(10, 6))
plt.plot(cvd_p, [lv * 100 for lv in liquid_dropout], 'b-o', linewidth=2, markersize=5)
plt.xlabel('Pressure [bara]', fontsize=12)
plt.ylabel('Retrograde Liquid Volume [%]', fontsize=12)
plt.title('CVD: Retrograde Liquid Dropout Curve', fontsize=14)
plt.grid(True, alpha=0.3)
plt.gca().invert_xaxis()
plt.tight_layout()
plt.show()

max_dropout = max(liquid_dropout) * 100
max_dropout_P = cvd_p[liquid_dropout.index(max(liquid_dropout))]
print(f"Maximum liquid dropout: {max_dropout:.1f}% at {max_dropout_P:.0f} bara")

## 6. Separator Test Simulation

Simulate a multi-stage separator train to determine stock tank oil properties.

In [ ]:
# Create fresh oil for separator test
oil_sep = oil.clone()
oil_sep.setTemperature(T_reservoir, "C")
oil_sep.setPressure(P_reservoir, "bara")
oil_sep.setTotalFlowRate(1.0, "mol/sec")  # Normalize to 1 mol/s basis

# Setup separator test
sep_test = jneqsim.pvtsimulation.simulation.SeparatorTest(oil_sep)

# Define separator stages
# Stage 1: HP separator
# Stage 2: LP separator  
# Stage 3: Stock tank
sep_pressures = [50.0, 10.0, 1.01325]  # bara
sep_temps = [60.0, 40.0, 15.0]  # °C

sep_test.setSeparatorConditions(sep_pressures, sep_temps, "bara", "C")

# Run
sep_test.runCalc()

print("Separator Test Results:")
print("=" * 50)
print(f"\nSeparator conditions:")
for i, (p, t) in enumerate(zip(sep_pressures, sep_temps)):
    print(f"  Stage {i+1}: {p:.1f} bara, {t}°C")

In [ ]:
# Get separator test results
try:
    Bo_sep = sep_test.getBo()
    Rs_sep = sep_test.getRs()
    GOR_sep = sep_test.getGOR()
    
    print(f"\nStock Tank Oil Properties:")
    print(f"  Bo (Formation Volume Factor): {Bo_sep:.4f} m³/Sm³")
    print(f"  Rs (Solution GOR): {Rs_sep:.1f} Sm³/Sm³")
    print(f"  Total GOR: {GOR_sep:.1f} Sm³/Sm³")
except:
    print("Note: Some separator test values may not be available")

## 7. Key PVT Parameters Summary

In [ ]:
print("""
PVT PARAMETERS SUMMARY
======================

Bo (Oil Formation Volume Factor)
  - Volume of reservoir oil / Volume of stock tank oil
  - Typical: 1.1 - 2.0 m³/Sm³
  - Increases with dissolved gas

Rs (Solution Gas-Oil Ratio)
  - Volume of dissolved gas / Volume of stock tank oil
  - Typical: 20 - 300 Sm³/Sm³ for black oils
  - Decreases as pressure drops below bubble point

Bg (Gas Formation Volume Factor)
  - Volume of reservoir gas / Volume of surface gas
  - Typically 0.003 - 0.03 m³/Sm³

Pb (Bubble Point Pressure)
  - Pressure where first gas bubble appears
  - Above Pb: undersaturated oil (single phase)
  - Below Pb: saturated oil (two phases)

Dew Point Pressure
  - Pressure where first liquid drop appears (gas condensates)
  - Below dew point: retrograde condensation
""")

## Summary

This notebook demonstrated:

1. **CCE (Constant Composition Expansion):**
   - Measures PV relationship and liquid dropout
   - Used to find bubble/dew point

2. **Differential Liberation:**
   - Simulates reservoir depletion for black oils
   - Provides Bo, Rs, oil density vs pressure

3. **CVD (Constant Volume Depletion):**
   - Simulates gas condensate reservoir depletion
   - Shows retrograde liquid dropout

4. **Separator Test:**
   - Optimizes separator conditions
   - Determines stock tank oil properties